In [62]:
using Pkg
Pkg.activate(joinpath(pwd(),".."))
using Random, Distributions, PrettyTables, Printf
Random.seed!(1234)

  Activating environment at `~/Desktop/ECON 627/Project.toml`


MersenneTwister(1234)

# Part (a): Data Generating

In [3]:
const β = 1
const pi = [1;1]
const ρ = 0.9

0.9

In [100]:
function gendata(n)
    #generate z1, z2
    mvnormal_z = MvNormal([0;0], [1 0 ; 0 1])
    Z = rand(mvnormal_z,n)'

    #generate ϵ, v and u
    mvnormal = MvNormal([0;0], [1 ρ ; ρ 1])
    err = rand(mvnormal, n)'

    e = err[:,1]
    V = err[:,2]

    U = (Z1 .+ Z2) .* e

    #generate X and y
    X = Z * pi + V
    Y = X * β + U

    return (Y = Y, X = X, Z = Z)
end


gendata (generic function with 1 method)

# Part (b): Efficient GMM Estimator

In [5]:
function est2SLS(y, x, z)
    # Set some parameter
    n = size(y,1)
    Q = (z'* x)/n
    W = inv(z' * z/n)

    #Beta
    beta2SLS = (Q' * W * Q)^(-1) * Q' * W * (z' * y)/n

    # Asymptotic variance and standard error  
    u = y - x * beta2SLS
    zu = z .* u
    Omega = (zu' * zu)/n

    var = (Q' * W * Q)^(-1) * Q' * W * Omega * W * Q * (Q' * W * Q)^(-1)
    
    sd2SLS = sqrt(var/n)


    return (beta2SLS=beta2SLS, sd2SLS = sd2SLS)
end

est2SLS (generic function with 1 method)

In [6]:
function estGMM(y,x,z)
    n=size(y,1)
    Q = (z'* x)/n
    W = inv(z' * z/n)

    #First Step
    (beta2SLS, sd) = est2SLS(y,x,z)

    #Second Step
    u = y - x * beta2SLS
    zu = z .* u
    Omega = (zu' * zu)/n
    betaGMM = (Q' * inv(Omega) * Q)^(-1) * Q' * inv(Omega) * (z' * y)/n

    avar = (Q' * inv(Omega) * Q)^(-1)  
    sdGMM = sqrt(avar)/sqrt(n)

    return (betaGMM =betaGMM, sdGMM = sdGMM)
end

estGMM (generic function with 1 method)

In [7]:
(Y,X,Z) = gendata(100)
estGMM(Y,X,Z)

(betaGMM = 0.9553209754408782, sdGMM = 0.13768093796415887)

# Part (c)

In [8]:
function stat(n)
    #generate data
    (y,x,z) = gendata(n)

    #compute weighting matrix
    (betaGMM, sdGMM) = estGMM(y,x,z)
    u = y - x * betaGMM
    zu = z .* u
    Omega = (zu' * zu)/n

    #compute J
    z1 = z[:,1]
    z2 = z[:,2]
    z1u = z1' * u
    z2u = z2' * u
    c = [z1u ; z2u]
    J = 1/n * c' * inv(Omega) * c

    return (Omega = Omega, J = J)
end

stat (generic function with 1 method)

In [99]:
(Omega, J) = stat(100)
reject = J > quantile(Chisq(1), .95) ? 1 : 0
println("The Efficient Weighting Matirx is ", Omega)
println("We should not reject the H0")

The Efficient Weighting Matirx is [0.3790574832482154 0.3149306592480312; 0.3149306592480312 0.33063494129476695]
We should not reject the H0


# Part (d)

In [10]:
function RejectionRate(n,R)
    result = zeros(R)
    for i = 1:R
        (Omega, J) = stat(n)
        result[i] = J > quantile(Chisq(1), .95) ? 1 : 0
    end
    rate = mean(result)
    return (rate = rate)
end

RejectionRate (generic function with 1 method)

In [12]:
rate = RejectionRate(100,10000)

0.0513

It is very close to 5\%

# Part (e) - (i)

$E[Z_i \epsilon_i] = E[Z_i U_i] + E[Z_i Z_i'] \delta = I_d \delta$

$E[Z_i X_i] = E[Z_i Z_i']\pi + E[Z_i V_i] = I_d \pi$



In [101]:
function gendata2(n, δ)
    mvnormal_z = MvNormal([0;0], [1 0 ; 0 1])
    Z = rand(mvnormal_z,n)'

    #generate ϵ, v and u
    mvnormal = MvNormal([0;0], [1 ρ ; ρ 1])
    err = rand(mvnormal, n)'

    e = err[:,1]
    V = err[:,2]

    U = (Z1 .+ Z2) .* e

    ϵ = U + Z * δ

    #generate X and y
    X = Z * pi + V
    Y = X * β + ϵ

    return (Y = Y, X = X, Z = Z)
end
    

gendata2 (generic function with 1 method)

In [18]:
function genstat2(n,δ)
    #generate data
    (y,x,z) = gendata2(n,δ)

    #compute weighting matrix
    (betaGMM, sdGMM) = estGMM(y,x,z)
    u = y - x .* betaGMM
    zu = z .* u
    Omega = (zu' * zu)/n

    #compute J
    z1 = z[:,1]
    z2 = z[:,2]
    z1u = z1' * u
    z2u = z2' * u
    c = [z1u ; z2u]
    J = 1/n * c' * inv(Omega) * c


    return (Omega = Omega, J = J)
end

genstat2 (generic function with 1 method)

In [19]:
function genRejectionRate2(n,R,δ)
    result2 = zeros(R)
    for i = 1:R
        (Omega, J) = genstat2(n,δ)
        result2[i] = J > quantile(Chisq(1), .95) ? 1 : 0
    end
    rate = mean(result2)
    return (rate = rate)
end

genRejectionRate2 (generic function with 1 method)

In [102]:
Δ = [0.2 1 20; -0.2 -1 20]
for i in 1:3
    δ = Δ[:,i]
    rate = genRejectionRate2(100, 10000, δ)
    println( δ,  "Rejection Rate is ", rate)
end


[0.2, -0.2]Rejection Rate is 0.7009


[1.0, -1.0]Rejection Rate is 1.0


[20.0, 20.0]Rejection Rate is 0.0515


The rejection probability for $\delta = [0.2, -0.2]$ and $\delta = [1, -1]$ are larger than previous case, however, when $\delta = [20,20]$, we have similar rejection probability. It is possible that for $\delta = k * [1,1]$, where $k$ is a positive constant, the asymptotic distribution of the J statistics is the same as the previous one.